### Days: wps_climdex_gsl

WPS wrapper for [climdex.pcic](https://cran.r-project.org/web/packages/climdex.pcic/climdex.pcic.pdf) spell duration index functions

- **Warm Spell Duration Index** ([climdex.wsdi](https://cran.r-project.org/web/packages/climdex.pcic/climdex.pcic.pdf#page=38)) The warm spell duration index is defined as the number of days each year which are part of a "warmspell".  A "warm spell" is defined as a sequence of 6 or more days in which the daily maximumtemperature exceeds the 90th percentile of daily maximum temperature for a 5-day running windowsurrounding this day during the baseline period.

In [1]:
import os
import requests
from birdy import WPSClient
from rpy2 import robjects
from urllib.request import urlretrieve
from pkg_resources import resource_filename
from tempfile import NamedTemporaryFile

from wps_tools.testing import get_target_url
from quail.utils import test_rda_output

In [2]:
# Ensure we are in the working directory with access to the data
while os.path.basename(os.getcwd()) != "quail":
    os.chdir('../')

In [3]:
# NBVAL_IGNORE_OUTPUT
url = get_target_url("quail")
print(f"Using quail on {url}")

Using quail on https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/quail/wps


In [4]:
quail = WPSClient(url)

#### Help for individual processes can be diplayed using the ? command (ex/ bird.process?)

In [5]:
# NBVAL_IGNORE_OUTPUT
quail.climdex_sdi?

Signature:
quail.climdex_sdi(
    climdex_input,
    ci_name,
    func=None,
    span_years=False,
    loglevel='INFO',
    output_file='output.rda',
    vector_name='days',
)
Docstring:
Cold or warm spell duration index

Parameters
----------
climdex_input : ComplexData:mimetype:`application/x-gzip`
    Rdata (.rda) file containing R Object of type climdexInput
ci_name : string
    Name of the climdexInput object
output_file : string
    Filename to store the output Rdata (extension .rda)
func : {'wsdi'}string
    Compute climdex.wsdi (Warm spell duration index)
span_years : boolean
    Specifies whether spells can cross year boundaries
vector_name : string
    Name to label the output vector
loglevel : {'CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG', 'NOTSET'}string
    Logging level

Returns
-------
rda_output : ComplexData:mimetype:`application/x-gzip`
    Rda file containing R output data
File:      ~/code/birds/quail/</tmp/quail-venv/lib/python3.8/site-packages/birdy/client/base

#### Run wps_climdex_sdi Process for climdex.wsdi() without spells spanning years

In [6]:
with NamedTemporaryFile(suffix=".rda", prefix="gsl_", dir="/tmp", delete=True) as output_file:
    output = quail.climdex_sdi(
            climdex_input=resource_filename("tests","data/climdexInput.rda"),
            ci_name="ci",
            vector_name="wsdi_vector",
            func="wsdi",
            output_file=output_file.name,
        )
wsdi_url = output.get()[0]

#### Run wps_climdex_sdi Process for climdex.wsdi() with spells spanning years

In [7]:
with NamedTemporaryFile(suffix=".rda", prefix="gsl_", dir="/tmp", delete=True) as output_file:
    output = quail.climdex_sdi(
            climdex_input=resource_filename("tests","data/climdexInput.rda"),
            ci_name="ci",
            vector_name="wsdi_span_yrs_vector",
            func="wsdi",
            span_years=True,
            output_file=output_file.name,
        )
wsdi_span_yrs_url = output.get()[0]

#### Test output against expected output

In [8]:
test_rda_output(
    wsdi_url, "wsdi_vector", "expected_sdi_data.rda", "expected_wsdi"
)
test_rda_output(
    wsdi_span_yrs_url, "wsdi_span_yrs_vector", "expected_sdi_data.rda", "expected_wsdi_span_years"
)